##### Imports and Utility Functions
Ensure the cell below is runs successfully to include all the helper utilities

In [2]:
%run workspaceutils

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 3, Finished, Available, Finished)

###### Check default lakehouse

In [3]:
if (mssparkutils.runtime.context['defaultLakehouseId']==None):
    displayHTML('<div style="display: flex; align-items: flex-end;"><img style="float: left; margin-right: 10px;" src="https://github.com/hurtn/images/blob/main/stop.png?raw=true" width="50"><span><h4>Please set a default lakehouse before proceeding</span><img style="float: right; margin-left: 10px;" src="https://github.com/hurtn/images/blob/main/stop.png?raw=true" width="50"></div>')
    print('\n')
    raise noDefaultLakehouseException('No default lakehouse found. Please add a lakehouse to this notebook.')
else: 
    print('Default lakehouse is set to '+ mssparkutils.runtime.context['defaultLakehouseName'] + '('+ mssparkutils.runtime.context['defaultLakehouseId'] + ')')

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 4, Finished, Available, Finished)

Default lakehouse is set to gitbranching(514f34f0-789f-43e9-b375-8383ad2560cb)


##### Store metadata of Capacities, Workspaces and Items

In [4]:
print(updateCp())
print(updateWs())
print(updateItems())
print(updateReports())

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 5, Finished, Available, Finished)

13 records saved to the capacities table.
43 records saved to the workspaces table.
Getting items for workspace golddemo...
Getting items for workspace VBD...
Getting items for workspace Admin monitoring...
Getting items for workspace Workspace1_for_BCDR...
Getting items for workspace DEV2...
Getting items for workspace F2Workspace...
Getting items for workspace oeiahack...
Getting items for workspace RTA Demo...
Getting items for workspace BCDR_SECONDARY...
Getting items for workspace dynamicsanalytics...
Getting items for workspace UAT...
Getting items for workspace Workspace2_for_BCDR_SECONDARY...
Getting items for workspace fabdemo...
Getting items for workspace Workspace2_for_BCDR...
Getting items for workspace DEV_SECONDARY...
Getting items for workspace DEV_Custom_Feature_01...
Getting items for workspace fnoanalytics...
Getting items for workspace DEV_Custom_Feature_99...
Getting items for workspace feature_dev_01...
Getting items for workspace SpireTest...
Getting items for wo

##### Save Workspace Git Connection Details

In [5]:
from pyspark.sql.functions import lit
verbose = False #flag to provide verbose error messages
client = fabric.FabricRestClient()

table_name = 'gitconnections'
spark.sql("Drop table if exists "+ table_name)
dfwks = spark.sql("SELECT distinct ID,Type,Name FROM workspaces where Type!='AdminInsights'").collect()
print("Storing git connection details for all workspaces.\nAny errors will appear below, some you can safely ignore such as the git connection status for the Capacity Metrics workspace...")
for idx,i in enumerate(dfwks):
    if i['Type'] == 'Workspace':
        url = "/v1/workspaces/" + i['ID'] + "/git/connection"
        try:
            print("Storing git details for workspace "+i['Name']+" ("+i['ID']+')...')
            response = client.get(url)

            gitProviderDetailsJSON = response.json()['gitProviderDetails']
            gitConnectionStateJSON = response.json()['gitConnectionState']
            gitSyncDetailsJSON = response.json()['gitSyncDetails']
            df = spark.createDataFrame([i['ID']],"string").toDF("Workspace_ID")
            df=df.withColumn("Workspace_Name",lit(i['Name'])).withColumn("gitConnectionState",lit(gitConnectionStateJSON)).withColumn("gitProviderDetails",lit(json.dumps(gitProviderDetailsJSON))).withColumn("gitSyncDetails",lit(json.dumps(gitSyncDetailsJSON)))

            if idx == 0:
                dfall = df
            else:
                dfall= dfall.union(df)
        except Exception as error:
            errmsg =  "Couldn't get git connection status for workspace " + i['Name'] + "("+ i['ID'] + ")."
            if (verbose):
                 errmsg = errmsg + "Error: "+str(error)
            print(str(errmsg))

dfall.withColumn("metaTimestamp",current_timestamp()).write.mode('overwrite').option("mergeSchema", "true").saveAsTable(table_name)
print('Done')



StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 6, Finished, Available, Finished)

Storing git connection details for all workspaces.
Any errors will appear below, some you can safely ignore such as the git connection status for the Capacity Metrics workspace...
Storing git details for workspace golddemo (c322b3f0-2900-45d0-b32f-cd68cf04ceba)...
Storing git details for workspace VBD (4d589ab3-677f-44e7-a5b1-9e2ecb7fcdd4)...
Storing git details for workspace Workspace1_for_BCDR (26c1b743-7cfa-4127-b469-785c3eaca6fd)...
Storing git details for workspace DEV2 (fec2c0b5-9fbb-462e-943e-2361c8e9b011)...
Storing git details for workspace F2Workspace (df3135fe-ed66-42e8-a4d8-889bc19d03f8)...
Storing git details for workspace oeiahack (e12e81a2-0a7d-4405-a7f9-bf3d67582f83)...
Storing git details for workspace RTA Demo (e17036c9-956f-42c1-89db-f9b9c798910b)...
Storing git details for workspace BCDR_SECONDARY (08b3eaba-381a-4f58-8f1f-b12c2254c62d)...
Storing git details for workspace dynamicsanalytics (372df74e-82a2-42ae-a2c2-5545ae5c6ee0)...
Storing git details for workspace U

In [6]:
mssparkutils.runtime.context

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 7, Finished, Available, Finished)

{'currentNotebookName': 'Create Feature Branch', 'defaultLakehouseName': 'gitbranching', 'defaultLakehouseId': '514f34f0-789f-43e9-b375-8383ad2560cb', 'parentRunId': None, 'defaultLakehouseWorkspaceId': 'fec2c0b5-9fbb-462e-943e-2361c8e9b011', 'hcReplId': None, 'activityId': '8e191ebe-4788-4960-af7b-1fdc40017024', 'currentWorkspaceId': 'fec2c0b5-9fbb-462e-943e-2361c8e9b011', 'referenceTreePath': None, 'currentNotebookId': '61c02a29-7e1f-4cc4-991c-57a63b0b0679', 'currentRunId': None, 'isForPipeline': False, 'rootRunId': None}

In [7]:
df = spark.sql("SELECT * FROM gitbranching.gitconnections where workspace_id = '"+ mssparkutils.runtime.context['currentWorkspaceId'] + "'")

gitsql = "select gt.gitConnectionState,gt.gitProviderDetails, wks.name Workspace_Name, wks.id Workspace_ID from gitconnections gt " \
         "inner join workspaces wks on gt.Workspace_Name = wks.name " \
         "where gt.gitConnectionState = 'ConnectedAndInitialized' and wks.id = '"+mssparkutils.runtime.context['currentWorkspaceId']+"'" 

dfgitdetails = spark.sql(gitsql).collect()
display(dfgitdetails)

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f58bbf4f-9fae-43a2-a100-16daf9143950)

In [8]:
df = spark.sql("SELECT * FROM gitbranching.capacities LIMIT 1000")
display(df)

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d192770-a6e2-4081-aed7-307318d97bf7)

In [9]:

df = spark.sql("SELECT capacity_id,name FROM gitbranching.workspaces where id = '"+ mssparkutils.runtime.context['currentWorkspaceId'] + "'").collect()
current_capacity_id = df[0][0]
current_ws_name = df[0][1]
print('Current capacity ID = '+current_capacity_id + ' for workspace ' + current_ws_name)

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 10, Finished, Available, Finished)

Current capacity ID = e62d671f-9ab0-4dab-a4ea-50c29a5239be for workspace DEV2


In [10]:
branch_to_new_ws = 'DEV_Custom_Feature_02'
branch_name = 'DEV_Custom_Feature_02_branch'
try:
    print("Creating workspace: " + branch_to_new_ws + " in capacity "+ current_capacity_id +"...")
    response = fabric.create_workspace(branch_to_new_ws,current_capacity_id) 
    new_workspace_id = response
    print("Created workspace with ID: " + new_workspace_id)
except Exception as error:
    errmsg =  "Failed to recreate workspace " +branch_to_new_ws + " with capacity ID ("+ current_capacity_id + ") due to: "+str(error)
    print(errmsg)

# Now update workspace table
updateWs()

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 11, Finished, Available, Finished)

Creating workspace: DEV_Custom_Feature_02 in capacity e62d671f-9ab0-4dab-a4ea-50c29a5239be...
Created workspace with ID: e0935818-7c12-40dc-902c-e7d79036242a


'45 records saved to the workspaces table.'

In [11]:
df = spark.sql("SELECT * FROM gitbranching.workspaces where name = '"+ branch_to_new_ws + "'")
display(df)

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 85858d39-97bd-4360-b637-71e9a787fbb3)

In [12]:
gitpayloadstr= '{"gitProviderDetails": ' + dfgitdetails[0][1] + '}'
print("Before: " + gitpayloadstr)
gitpayload = json.loads(gitpayloadstr)
gitpayload["gitProviderDetails"]["branchName"] = branch_name
print("After: " +json.dumps(gitpayload))

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 13, Finished, Available, Finished)

Before: {"gitProviderDetails": {"organizationName": "hurtn", "projectName": "DEV2", "gitProviderType": "AzureDevOps", "repositoryName": "DEV2", "branchName": "main", "directoryName": "/Fabric"}}
After: {"gitProviderDetails": {"organizationName": "hurtn", "projectName": "DEV2", "gitProviderType": "AzureDevOps", "repositoryName": "DEV2", "branchName": "DEV_Custom_Feature_02_branch", "directoryName": "/Fabric"}}


In [13]:
url = "/v1/workspaces/" +  new_workspace_id + "/git/connect"

try:
    print('Attempting to connect workspace '+ branch_to_new_ws)
    response = client.post(url,json= gitpayload)
    print(str(response.status_code) + response.text) 
    success = True
    
except Exception as error:
    errmsg =  "Couldn't connect git to workspace " + branch_to_new_ws + ". Error: "+str(error)
    print(str(errmsg))
    success = False
# If connection successful then try to initialise    
if (success):
    url = "/v1/workspaces/" + new_workspace_id + "/git//initializeConnection"
    payload = {"initializationStrategy":"PreferRemote"}
    try:
        print('Attempting to initialize git connection for workspace '+ branch_to_new_ws)
        response = client.post(url,json= payload)
        #print(str(response.status_code) + response.text) 
        commithash = response.json()['remoteCommitHash']
        print('Successfully initialized. Updating with commithash '+commithash)
        if commithash!='':
            url = "/v1/workspaces/" + new_workspace_id + "/git/updateFromGit"
            payload = '{"remoteCommitHash": "' + commithash + '","conflictResolution": {"conflictResolutionType": "Workspace","conflictResolutionPolicy": "PreferWorkspace"},"options": {"allowOverrideItems": true}}'
            response = client.post(url,json= json.loads(payload))
            print(str(response.status_code))
    except Exception as error:
        errmsg =  "Couldn't initialize git for workspace " +branch_to_new_ws + ". Error: "+str(error)
        print(str(errmsg))

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 14, Finished, Available, Finished)

Attempting to connect workspace DEV_Custom_Feature_02
200
Attempting to initialize git connection for workspace DEV_Custom_Feature_02
Successfully initialized. Updating with commithash 209108904f0ffe12e91b0d5954923a8e1e21689b
202


In [14]:
df = spark.sql("SELECT * FROM gitbranching.workspaces LIMIT 1000")
display(df)

StatementMeta(, 8e191ebe-4788-4960-af7b-1fdc40017024, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2de8c7d2-4ede-4944-9e0e-0bba6140ca43)